In [17]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series

In [18]:
# import .csv files as dataframes 
v6_GISInfo = pd.read_csv('input/v6_GISInfoTable_2016-08-09.csv', low_memory=False)
v6_Main_only = pd.read_csv('input/MainTable_and_not_GISInfoTable.csv', low_memory=False)

In [33]:
# function for exporting dataframes to .csv files, for progress-checking
def progress_check(df, name):
    df.to_csv('TESTING_%s.csv' % name)

In [21]:
# append all the rows that had IDs in MainTable that aren't in the original GISInfoTable
# Rule: where bou_id and pt_id are the same, if x&y are empty or =0s, use bou_id and set type to POLYGON
#           ...if x&y !=0, use PT_ID and set as POINT object types

# generating DataFrame of "normal" ('BOU_ID'==0), point items from MainTableOnly
mask_normal = v6_Main_only['BOU_ID'] == 0
Main_normal = v6_Main_only[mask_normal]
Main_normal['orig_id'] = Main_normal['PT_ID']
Main_normal['obj_type'] = 'POINT'

# preparing DataFrame of "abnormal" ('BOU_ID' == 'PT_ID') rows from MainTableOnly
mask_abnormal = v6_Main_only['BOU_ID'] == v6_Main_only['PT_ID']
Main_abnormal = v6_Main_only[mask_abnormal]

# generating DataFrame of polygon rows from abnormal rows from MainTableOnly
# HAVE TO MASK 
mask_polygons = Main_abnormal['X_COOR'] == 0
Main_abnormal_polygons = Main_abnormal[mask_polygons]
Main_abnormal_polygons['orig_id'] = Main_abnormal_polygons['BOU_ID']
Main_abnormal_polygons['obj_type'] = 'POLYGON'

# generating DataFrame of point rows from abnormal rows from MainTableOnly
# TODO: SEE IF THIS CAN BE DRY-ED OUT AND COMBINED WITH THE ABOVE Main_normal STUFF
mask_points = Main_abnormal['X_COOR'] != 0
Main_abnormal_points = Main_abnormal[mask_points]
Main_abnormal_points['orig_id'] = Main_abnormal_points['PT_ID']
Main_abnormal_points['obj_type'] = 'POINT'


#### Below is old as of 160813 11pm

# masking for selecting rows in MainTable that have non-zero BOU_ID and PT_ID
#mask_BOU_ID = v6_Main_only['BOU_ID']!=0
#v6_Main_issues = v6_Main_only[mask_BOU_ID]
#mask_PT_ID = v6_Main_issues['PT_ID']!=0
#v6_Main_issues = v6_Main_issues[mask_PT_ID]


# CAN TOSS THIS CHUNK
#print(v6_Main_issues)
#v6_Main_only_masked = v6_Main_only[mask_BOU_ID]
#mask_two_ID = v6_Main_only_masked['PT_ID']!= 0
#v6_Main_only_masked = v6_Main_only_masked[mask_two_ID]


# masking for distinct cases
#mask_polygon = v6_Main_issues['X_COOR'] == 0
#mask_point = v6_Main_issues['X_COOR'] != 0

# editing columns via masks for distinct cases
#Main_polygons = v6_Main_issues[mask_polygon]
#Main_polygons['orig_id'] = Main_polygons['BOU_ID']
#Main_polygons['']

#Main_points = v6_Main_issues[mask_point]
#Main_

#print(Main_polygons)

#Main_polygons['sys_id'] = "hvd_%d" % Main_polygons['orig_id']


/home/sf/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/sf/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/sf/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.h

In [25]:
# merging back together processed parts of the MainOnlyTable
v6_Main_only_processed = pd.concat([Main_normal, Main_abnormal_polygons, Main_abnormal_points])
# print(v6_Main_only_processed) 
# LOOKS GOOD

      Unnamed: 0           NAME_PY NAME_CH NAME_FT       PRES_LOC BEG_CHG_TY  \
2          12682        Guilin Jun     桂林郡     桂林郡    今广西桂平县西南白沙镇         新建   
3          12683        Guilin Jun     桂林郡     桂林郡    今广西桂平县西南白沙镇       辖区缩小   
4          12684         Xiang Jun      象郡      象郡         今广西崇左县         新建   
5          12685         Xiang Jun      象郡      象郡         今广西崇左县       辖区缩小   
6          12686         Yulin Jun     郁林郡     郁林郡    今广西桂平县西南白沙镇         新建   
7          12687         Yulin Jun     郁林郡     郁林郡    今广西桂平县西南白沙镇       辖区扩大   
8          12688        Yuping Jun     郁平郡     郁平郡    今广西桂平县西南白沙镇         更名   
9          12689         Yulin Jun     郁林郡     郁林郡    今广西桂平县西南白沙镇         更名   
10         12690         Yulin Jun     郁林郡     郁林郡    今广西桂平县西南白沙镇       辖区缩小   
11         12691         Yulin Jun     郁林郡     郁林郡    今广西桂平县西南白沙镇       辖区缩小   
12         12692         Yulin Jun     郁林郡     郁林郡    今广西桂平县西南白沙镇       辖区扩大   
13         12693        Guilin Jun     桂

In [34]:
# Adding 'sys_id' field that consists of 'hvd_' plus 'orig_id' value
v6_Main_only_processed['sys_id'] = 'hvd_' + v6_Main_only_processed['orig_id'].astype(str)
progress_check(v6_Main_only_processed, '160813_experiment') # PASS

In [39]:
#### NEEDS SOME WORK HERE; EVERYTHING ABOVE SEEMS FINE (160813 END OF DAY)

# joining together GISInfoTable and v6_Main_only_processed
v6_all = pd.concat([v6_GISInfo, v6_Main_only_processed])
#v6_all = v6_GISInfo.join(v6_Main_only_processed, on='sys_id', how='inner')
progress_check(v6_all, 'v6_all')

In [ ]:
# rename existing GISInfoTable col named "sys_id" to "orig_id", create new col that is "hvd_%d" % <orig_id>
#v6_GISInfo

In [ ]:
# name the result of the union of the v6 GISInfoTable and MainTable "V6_input_draft_20160811.csv"